# VGG Network

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds


## Create Named Variables Dynamically

In [5]:
# Define a small class MyClass
class MyClass(tf.keras.Model):
    def __init__(self):
        # One class variable 'a' is set to 1
        self.var1 = 1

# Create an object of type MyClass()
my_obj = MyClass()

In [6]:
my_obj.__dict__

{'_self_setattr_tracking': True,
 '_obj_reference_counts_dict': ObjectIdentityDictionary({<_ObjectIdentityWrapper wrapping 1>: 1}),
 'var1': 1}

In [7]:
vars(my_obj)

{'_self_setattr_tracking': True,
 '_obj_reference_counts_dict': ObjectIdentityDictionary({<_ObjectIdentityWrapper wrapping 1>: 1}),
 'var1': 1}

In [8]:
# Call vars, passing in the object.  Then access the __dict__ dictionary using square brackets
vars(my_obj)['var3'] = 3

# Call vars() to see the object's instance variables
vars(my_obj)

{'_self_setattr_tracking': True,
 '_obj_reference_counts_dict': ObjectIdentityDictionary({<_ObjectIdentityWrapper wrapping 1>: 1}),
 'var1': 1,
 'var3': 3}

In [9]:
# Use a for loop to increment the index 'i'
for i in range(4,10):
    # Format a string that is var
    vars(my_obj)[f'var{i}'] = 0
    
# View the object's instance variables!
vars(my_obj)

{'_self_setattr_tracking': True,
 '_obj_reference_counts_dict': ObjectIdentityDictionary({<_ObjectIdentityWrapper wrapping 1>: 1}),
 'var1': 1,
 'var3': 3,
 'var4': 0,
 'var5': 0,
 'var6': 0,
 'var7': 0,
 'var8': 0,
 'var9': 0}

In [10]:
i = 1
print(f"var{i}")

var1


In [11]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        vars(self)['var1'] = 1
        
# Create an object of type MyClass()
my_obj = MyClass()
vars(my_obj)

{'var1': 1}

## Create a Generic VGG Block

In [12]:
class Block(tf.keras.Model):
    
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1 etc based on the number of repetition
        for i in range(0, repetitions):
            # Define a Conv2D layer
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(
                self.filters,
                kernel_size=kernel_size,
                strides=strides,
                activation='relu',
                padding='same'
            )
            
        # Define the max pool layer that will be added after the conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D((pool_size, pool_size))
        
        
    def call(self, inputs):
        # Access the class's conv2D_0 layer
        conv2D_0 = self.conv2D_0
        
        x = conv2D_0(inputs)
        
        # for the remaining conv2D_i layers from 1 to repetitions 
        # they will be connected to thre previous lauer
        for i in range(1, self.repetitions):
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            x = conv2D_i(x)
            
        # Finally add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

## Create Custom VGG Network

In [ ]:
class MyVGG(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyCGG, self).__init__()
        
        # Creating blocks of VGG with the following
        # 
        self.block_a = Block(64, kernel_size=3, repetitions=2)
        self.block_b = Block(128, kernel_size=3, repetitions=2)
        self.block_c = Block(256, kernel_size=3, repetitions=3)
        self.block_d = Block(512, kernel_size=3, repetitions=3)
        self.block_e = Block(512, kernel_size=3, repetitions=3)
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
        
        
    def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        
        return x